In [1]:
import pandas as pd
import numpy as np
import glob, os, sys
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='notebook', style = 'whitegrid', font_scale = 1.3)

In [2]:
sys.path.append('..')
from modules.plotting_metrics import PlotMetric

In [19]:
prot_name = 'cdk2'
# Este es el DF que no tiene aun afgregados los valores AUC de las conformaciones
path_to_json_file = glob.glob((os.path.join('..', 'data', 
                        F'TABLA_MTDATA_{prot_name.upper()}_*_crys_LIGS_INFO_LABELS.json')))[0]
                      
df_prot = pd.read_json(path_to_json_file)

In [20]:
# Extract just the interesting features
df_prot.columns
columns_ = ['Labels_conf', 'Inhib', 'Inhib_mass']
df_CDK2 = df_prot[columns_]

#### MDS info

In [4]:
# Read the MDS resutls
import pickle

# Pocket MDS
path_mds_obj_pk = os.path.join('..', 'data', 'trajectory_analysis', 'cMDS_Pocket_402_obj.pyobj')
with open(path_mds_obj_pk, 'rb') as f:
    mds_pocket_402 = pickle.load(f)
    
# Pisani MDS
path_mds_obj = os.path.join('..', 'data', 'trajectory_analysis', 'cMDS_Pisani_402_obj.pyobj')
with open(path_mds_obj, 'rb') as f:
    mds_pisani_402 = pickle.load(f)
    
a = mds_pisani_402[0][:2]
b = mds_pocket_402[0][:2]

c = np.concatenate((a, b))

np.savetxt('./mds_subspaces_pis_pk.csv', c, delimiter = ',')

In [6]:
# Helper function
%run ./helper_functions_S6.ipynb

In [15]:
score_type = 'Dk_lef'
df_swarm_dklef = pd.DataFrame()

for db_name in scores_dic:
    _X = scores_dic[db_name][score_type]['X']
    _y = scores_dic[db_name][score_type]['y']
    auc_scores = _X.apply(lambda x: roc_auc_score(y_true =_y, 
                                  y_score = -1 * x), axis = 0)
    score_type_ = score_type.lower().replace('_', '')
    df_swarm_dklef[db_name + '_vrd_' + score_type_] = auc_scores

score_type = 'Dk_sc'
df_swarm_dksc = pd.DataFrame()

for db_name in scores_dic:
    _X = scores_dic[db_name][score_type]['X']
    _y = scores_dic[db_name][score_type]['y']
    auc_scores = _X.apply(lambda x: roc_auc_score(y_true =_y, 
                                  y_score = -1 * x), axis = 0)
    score_type_ = score_type.lower().replace('_', '')
    df_swarm_dksc[db_name + '_vrd_' + score_type_] = auc_scores
    

In [25]:
x = pd.concat([df_CDK2, df_swarm_dksc, df_swarm_dklef], axis = 1)

In [26]:
x.to_csv('./df_pdb_cdk2_features_and_scores.csv')

### Consensus Scorings

In [ ]:
# kmeans or random
# Consensus type
# Docking score
# Database

# Save each file

In [2]:
from glob import glob
import pickle
import numpy as np
import pandas as pd

In [3]:
path_ = '../data/ml_evaluations/consensus_scoring/'

# Each final csv goint to have the 
# consensus values, and databases



In [17]:
methods = ('kmeans', 'rfe')
cons_list_names = ['Rbn', 'Rbs', 'Rbr', 'Rbv_*2', 'Rexp*2']
inner_keys = ['pisani', 'pocket']

final_list = []

for method in methods:
    for cons in cons_list_names:
        file = glob(F'{path_}*{method}*{cons}*')[0]

        with open(file, 'rb') as f:
            dict_auc = pickle.load(f)
        # iterate over the keys
        for key in dict_auc.keys():
            # for kmeans we currently have pisani and pocket
            for in_key in inner_keys:
                if method == 'kmeans':
                    _in_key = '-' + in_key.lower()
                    list_values = dict_auc[key][in_key].tolist()
                elif method == 'rfe':
                    _in_key = ''
                    list_values = dict_auc[key]['rfe'].tolist()
                names = key.split('_')
                # Get the databaseName
                db = names[0]
                # Get the score type
                score = names[1]

                # Append new elemnts at the begining of the list
                # Scoring > MethodFeature > DB > Consensus > Values
                list_values.insert(0, cons.lower())
                list_values.insert(0, db.lower())
                list_values.insert(0, (method + _in_key).lower())
                list_values.insert(0, score.lower())
                final_list.append( list_values )
                if method == 'kmeans': continue
                else: break
# Save the file
file_name = './kmeans_cosensus_scorings.csv'
df_ = pd.DataFrame(final_list).round(3)
df_.to_csv(file_name)